<a href="https://colab.research.google.com/github/nklsrmy/CC_MACLRL_EXERCISES_COM222ML/blob/main/Exercise9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 9: Choosing the best performing model on a dataset

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
- Use all classification models

Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e10/overview

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

## Dataset File

In [2]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/train.csv?raw=true'
df = pd.read_csv(dataset_url)

## Test File

In [3]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [4]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39098 non-null  int64  
 1   person_age                  39098 non-null  int64  
 2   person_income               39098 non-null  int64  
 3   person_home_ownership       39098 non-null  object 
 4   person_emp_length           39098 non-null  float64
 5   loan_intent                 39098 non-null  object 
 6   loan_grade                  39098 non-null  object 
 7   loan_amnt                   39098 non-null  int64  
 8   loan_int_rate               39098 non-null  float64
 9   loan_percent_income         39098 non-null  float64
 10  cb_person_default_on_file   39098 non-null  object 
 11  cb_person_cred_hist_length  39098 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.6+ MB


## Sample Submission File

In [5]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/sample_submission.csv?raw=true'
sf=pd.read_csv(sample_submission_url)

In [6]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           39098 non-null  int64  
 1   loan_status  39098 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 611.0 KB


In [7]:
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [8]:
df.describe()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,58645.000000,58645.000000,5.864500e+04,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000
mean,29322.000000,27.550857,6.404617e+04,4.701015,9217.556518,10.677874,0.159238,5.813556,0.142382
std,16929.497605,6.033216,3.793111e+04,3.959784,5563.807384,3.034697,0.091692,4.029196,0.349445
min,0.000000,20.000000,4.200000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.000000
25%,14661.000000,23.000000,4.200000e+04,2.000000,5000.000000,7.880000,0.090000,3.000000,0.000000
50%,29322.000000,26.000000,5.800000e+04,4.000000,8000.000000,10.750000,0.140000,4.000000,0.000000
75%,43983.000000,30.000000,7.560000e+04,7.000000,12000.000000,12.990000,0.210000,8.000000,0.000000
max,58644.000000,123.000000,1.900000e+06,123.000000,35000.000000,23.220000,0.830000,30.000000,1.000000


In [9]:
df['loan_status'].value_counts()

,count
loan_status,
0,50295
1,8350


In [10]:
# Filling missing values for categorical columns
for column in df.select_dtypes(include='object').columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

# Filling missing values for numerical columns
for column in df.select_dtypes(include='number').columns:
    df[column].fillna(df[column].mean(), inplace=True)

# Repeat the same for test data
for column in dt.select_dtypes(include='object').columns:
    dt[column].fillna(dt[column].mode()[0], inplace=True)

for column in dt.select_dtypes(include='number').columns:
    dt[column].fillna(dt[column].mean(), inplace=True)

<ipython-input-10-7b30006ae766>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace=True)
<ipython-input-10-7b30006ae766>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [11]:
df.isnull().sum()

,0
id,0
person_age,0
person_income,0
person_home_ownership,0
person_emp_length,0
loan_intent,0
loan_grade,0
loan_amnt,0
loan_int_rate,0
loan_percent_income,0


In [12]:
X = df.drop(columns=['loan_status'])  # Features
y = df['loan_status']

# One-hot encoding for categorical variables
X = pd.get_dummies(X)
dt = pd.get_dummies(dt)

# Ensure both datasets have the same dummy variables
X, dt = X.align(dt, join='left', axis=1, fill_value=0)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Calculate the correlation matrix
corr_matrix = X.corr().abs()

# Select the upper triangle of the correlation matrix
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find the columns that have a correlation greater than 0.95
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]

# Drop the columns with high correlation from X
X = X.drop(columns=to_drop)

# Print the dropped columns
print(f"Dropped columns due to high correlation (> 0.95): {to_drop}")

Dropped columns due to high correlation (> 0.95): ['cb_person_default_on_file_Y']


## 1. Train a KNN Classifier

In [14]:
knn = KNeighborsClassifier(n_neighbors=13)
knn.fit(X, y)

KNeighborsClassifier(n_neighbors=13)

- Perform cross validation

In [15]:
knn_scores = cross_val_score(knn, X, y, cv=5)

## 2. Train a Logistic Regression Classifier

In [16]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

- Perform cross validation

In [17]:
lr_scores = cross_val_score(lr, X, y, cv=5)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

## 3. Train a Naive Bayes Classifier

In [18]:
nb = GaussianNB()
nb.fit(X, y)

GaussianNB()

- Perform cross validation

In [19]:
nb_scores = cross_val_score(nb, X, y, cv=5)

## 4. Train a SVM Classifier

In [20]:
svm = SVC()
svm.fit(X, y)

SVC()

- Perform cross validation

In [21]:
svm_scores = cross_val_score(svm, X, y, cv=5)

## 5. Train a Decision Tree Classifier

In [22]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

- Perform cross validation

In [23]:
dtc_scores = cross_val_score(dtc, X, y, cv=5)

## 6. Train a Random Forest Classifier

In [24]:
rf = RandomForestClassifier()
rf_scores = cross_val_score(rf, X, y, cv=5)

## 7. Compare all the performance of all classification models

In [25]:
# Calculate the mean of the cross-validation scores (ensuring they exist)
knn_accuracy = np.mean(knn_scores) if len(knn_scores) > 0 else None
svm_accuracy = np.mean(svm_scores) if len(svm_scores) > 0 else None
lr_accuracy = np.mean(lr_scores) if len(lr_scores) > 0 else None
nb_accuracy = np.mean(nb_scores) if len(nb_scores) > 0 else None
dtc_accuracy = np.mean(dtc_scores) if len(dtc_scores) > 0 else None
rf_accuracy = np.mean(rf_scores) if len(rf_scores) > 0 else None

# Store the accuracies in the dictionary if they are not None
model_performance = {}

if knn_accuracy is not None:
    model_performance['KNN'] = knn_accuracy
if svm_accuracy is not None:
    model_performance['SVM'] = svm_accuracy
if lr_accuracy is not None:
    model_performance['Logistic Regression'] = lr_accuracy
if nb_accuracy is not None:
    model_performance['Naive Bayes'] = nb_accuracy
if dtc_accuracy is not None:
    model_performance['Decision Tree'] = dtc_accuracy
if rf_accuracy is not None:
    model_performance['Random Forest'] = rf_accuracy

# Check if the dictionary is populated
if not model_performance:
    print("Error: No models have been evaluated.")
else:
    # Print performance of all models
    print("Model Performance (Average CV Accuracy):")
    for model, score in model_performance.items():
        print(f"{model}: {score:.4f}")

    # Find and print the best model
    best_model_name = max(model_performance, key=model_performance.get)
    best_model_score = model_performance[best_model_name]
    print(f"\nBest Model: {best_model_name} with Accuracy: {best_model_score:.4f}")

Model Performance (Average CV Accuracy):
KNN: 0.8727
SVM: 0.8576
Logistic Regression: 0.9015
Naive Bayes: 0.8847
Decision Tree: 0.7566
Random Forest: 0.8829

Best Model: Logistic Regression with Accuracy: 0.9015


## 9. Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [26]:
# If 'id' column is missing, generate a default index
id_column = np.arange(1, len(dt) + 1)  # Create a default index starting from 1
y_pred = dtc.predict(dt)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': id,  # Use the generated index as 'id'
    'loan_status': y_pred
})

# Save to CSV
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
